# ResNet Demo

CS/Math 667 Fall 2018
Presented Dec. 3, 2018

This demo is using Tensorflow, Docker, Jupyter Notebook, and Imagenet Classes.

The requirements.txt, jupyter notebook, Docker Hub link, and example images will be shared so that you may run this example and/or input your own images or url's for classification. 

Note: Local images can be of any type supported by PIL && matplotlib.

Note: URL's can only be jpegs.

The model used:
(https://storage.googleapis.com/download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v2_fp32_savedmodel_NHWC_jpg.tar.gz)


# Let's begin

To start we need the Docker image: 
more info may be found here https://www.tensorflow.org/serving/
in a shell: 
```bash
docker pull tensorflow/serving@sha256:1aaf111b4abb9f2aee618d13f556ab24fee4fff4c44993683772643a7c513b1d
```

# ImageNet classes: 
https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
Note: imagenet_class_index.json is bundled with this notebook for your convenience. 


# Python virtual environment:
in a shell:
```bash
python3 -m venv tf_client_venv
. ./tf_client_venv/bin/activate
pip install -r requirements.txt
```


# ResNet Model:
https://storage.googleapis.com/download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v2_fp32_savedmodel_NHWC_jpg.tar.gz

And extract into your homedirectory
```bash
mkdir -p ~/tmp/resnet
curl -s http://download.tensorflow.org/models/official/20181001_resnet/savedmodels/resnet_v2_fp32_savedmodel_NHWC_jpg.tar.gz | \
tar --strip-components=2 -C $HOME/tmp/resnet -xvz
```

# Jupyter Notebook
This resnet_demo is a Python 3 Juptyer notebook.

# Input image or url (jpeg)
A few sample images are included with this notebook.




# Now we can start the TensorFlow server and open Jupyter Notebook.

We started the virtual environment in the previous cell 
 with the following:
```bash
. ./tf_client_venv/bin/activate
```

# Next, we can start the Docker tensorflow/serving image 
The order of activating the virtual environment or Docker does not matter.
However, the virtual environment needs to be activated before 
running jupyter-notebook.
Note: the following command is one line:
```bash
      docker run --rm -p 8501:8501 --name tfserving_resnet \
      --mount type=bind,source=$HOME/tmp/resnet,target=/models/resnet -e \
      MODEL_NAME=resnet -t tensorflow/serving &
```

# Now we load Jupyter Notebook:
```bash 
jupter-notebook
```

# Open the resnet_demo.ipynb
You may use the demo to classify your own images. 

Just replace the input to classify_image() with a local image path:
```bash
      classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/lemon.jpg')
```

or with a url to a jpeg
```bash
      classify_image('https://www.example.com/lemon.jpg')
```


In [ ]:
import base64
import json
import requests
from PIL import Image
from matplotlib.pyplot import imshow, imread
import numpy as np
from urllib3.util import url
from urllib.request import urlopen
import io
%matplotlib inline


# The model being used is 
# https://storage.googleapis.com/download.tensorflow.org/models/official/20181001_resnet/
#    savedmodels/resnet_v2_fp32_savedmodel_NHWC_jpg.tar.gz


In [ ]:
# Image file reading and classifying
#
#
# Images need to be read and then encoded into base64 
# and then placed into a JSON object (as per API).
# Python will do this for you if you ask nicely.

# Note: The model dictates how the input should be formatted. 
# So, it is super important to know what the format is in order
# to feed it information to classify.



# Encode input image into base64 and display the image


def display_and_load(filename):
    uri = url.parse_url(filename)
    if uri.scheme is None:
        with open(filename, 'rb') as fd:
            encoded_image = base64.b64encode(fd.read()).decode('utf8')
        image = imread(filename)
    else:
        with urlopen(filename) as conn:
            data = io.BytesIO(conn.read())
            encoded_image = base64.b64encode(data.read()).decode('utf8')
            data.seek(0)
        image = imread(data, 'jpg')

    imshow(image)
    return encoded_image



# Loads the ImageNet classes dictionary

with open("/home/linda/deep_residual_learning_project/imagenet_class_index.json") as fd:
    classes_dictionary = json.load(fd)

    
    
# Classifies the input image (local file or jpeg url)

def classify_image(filename):
    encoded_string = display_and_load(filename)
    data = {'instances': [{'b64': encoded_string}]}
    r = requests.post('http://localhost:8501/v1/models/resnet:predict', json = data)
    predicted_class = r.json()['predictions'][0]['classes']
    prediction = classes_dictionary[str(predicted_class - 1)]
    print('The predicted class is: {}: {}'.format(predicted_class - 1, prediction)) 


In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/lemon.jpg')

In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/l0.jpg')

In [ ]:
classify_image('https://farm4.staticflickr.com/3218/3054852015_ecb7c51d37_m_d.jpg')

In [ ]:
classify_image('https://farm1.staticflickr.com/173/470851368_85fa244783_m_d.jpg')

In [ ]:
classify_image('https://farm1.staticflickr.com/153/409287804_543c529cb3_m_d.jpg')

In [ ]:
classify_image('https://farm1.staticflickr.com/128/394938875_817edf4eba_m_d.jpg')

In [ ]:
classify_image('https://farm1.staticflickr.com/105/271302810_6d3c76ed3d_m_d.jpg')

In [ ]:
classify_image('https://farm1.staticflickr.com/46/139492445_6e3b74f881_z_d.jpg')

So, now we have seen one local image and several creative commons images. 

How about we try to classify some new input(s)?

Note: For the live demo, our team showed objects in person and asked the audience to classify 
the objects and then we asked the model to classify the objects and compared the results.


In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/r0.jpg')

In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/p1.jpg')

In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/j1.jpg')

In [ ]:
classify_image(os.path.join(os.environ['HOME'],'/tmp/resnet_demo/l1.jpg')

# Hrmmm... 

looks like we need to see about getting this book's cover into the labelled dataset.

Ok, just one more image to classify.

In [ ]:
classify_image('https://upload.wikimedia.org/wikipedia/commons/f/f9/STOP_sign.jpg')

# Yay, classifying is fun!

Now you are ready to use this ResNet demo to classify your own images.

# This demo was made possible by open source software and creative commons images. 

# Thanks to all of the open source and creative commons contributors. 